In [1]:
from tqdm import tqdm
import cv2
import numpy as np

from utils import list_images
from yunet import YuNet
from sface import SFace

You can download the LFW dataset here:
http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
http://vis-www.cs.umass.edu/lfw/#download

in this dataset we are going to have some duplicate samples, but even with these duplicate data you can clearly see tht the threshold is high enought.

In [2]:
IMAGES_PATH = "/home/rodrigo/Downloads/lfw-deepfunneled"
USE_YUNET = True
THRESHOLD = 0.363  # If we use something above 0.55, the rate of false positive will be below 2%

In [3]:
weights_path = "./sface/bins/face_recognition_sface_2021dec.onnx"
recognizer = SFace(model_path=weights_path, dis_type=0, backend_id=cv2.dnn.DNN_BACKEND_CUDA,
                   target_id=cv2.dnn.DNN_TARGET_CUDA)

face_detector = YuNet()

In [4]:
images_path = list_images(IMAGES_PATH)

hashs = []
for fname in tqdm(list(images_path)):
    image = cv2.imread(fname)
    if USE_YUNET:
        try:  # not all images are going to find some face
            data = face_detector.detect(image)[0][1]
            face_hash = recognizer.infer(image, data)
        except:
            continue
    else:
        face_hash = recognizer.infer(image)
    hashs.append({"hash": cv2.normalize(face_hash, None), "fname": fname})

100%|██████████| 5748/5748 [01:46<00:00, 54.18it/s]


In [5]:
dist_mat = np.zeros((len(hashs), len(hashs)))
for i in tqdm(range(len(hashs))):
    for j in range(len(hashs)):
        if i == j:  # we don't want the same image to be compared with itself
            continue
        dist_mat[i, j] = (hashs[i]["hash"] * hashs[j]["hash"]).sum()  # cosine distance

100%|██████████| 5748/5748 [01:14<00:00, 77.57it/s]


In [6]:
# get best match
x, y = np.unravel_index(dist_mat.argmax(), dist_mat.shape)

print(hashs[x]["fname"])
print(hashs[y]["fname"])
print(x, y, dist_mat.max())

cv2.imshow("image", cv2.imread(hashs[x]["fname"]))
cv2.imshow("image2", cv2.imread(hashs[y]["fname"]))
cv2.waitKey(0)
cv2.destroyAllWindows()

/home/rodrigo/Downloads/lfw-deepfunneled/Eva_Amurri/Eva_Amurri_0001.jpg
/home/rodrigo/Downloads/lfw-deepfunneled/Emmy_Rossum/Emmy_Rossum_0001.jpg
281 4339 0.9704052209854126


In [7]:
# does the same match with the opencv detector, just to compare if the results are the same
recognizer._model.match(hashs[x]["hash"], hashs[y]["hash"])

0.9704051511271246

In [8]:
# how many false positives are there?
rank = {}
matches = {}
for i in range(dist_mat.shape[0]):
    for j in range(dist_mat.shape[1]):
        if dist_mat[i, j] > THRESHOLD + 0.2:
            if i not in rank:
                rank[i] = 1
                matches[i] = [j]
            else:
                rank[i] += 1
                matches[i].append(j)

keys = list(rank.keys())

print("False positives:", len(keys), len(keys) / len(hashs))  # We suppose that there are only picture for each person

False positives: 90 0.015657620041753653


In [9]:
# Which sample has more false positives?
y = dict(sorted(rank.items(), key=lambda x: x[1], reverse=True))
y

{63: 1,
 89: 1,
 144: 1,
 185: 1,
 281: 1,
 282: 1,
 296: 1,
 420: 1,
 529: 1,
 618: 1,
 637: 1,
 750: 1,
 786: 1,
 797: 1,
 804: 1,
 882: 1,
 897: 1,
 920: 1,
 927: 1,
 1019: 1,
 1052: 1,
 1156: 1,
 1201: 1,
 1227: 1,
 1293: 1,
 1428: 1,
 1436: 1,
 1484: 1,
 1530: 1,
 1607: 1,
 1622: 1,
 1699: 1,
 1785: 1,
 1789: 1,
 1884: 1,
 1949: 1,
 2043: 1,
 2060: 1,
 2113: 1,
 2126: 1,
 2168: 1,
 2365: 1,
 2399: 1,
 2499: 1,
 2589: 1,
 2637: 1,
 2755: 1,
 2858: 1,
 2880: 1,
 2885: 1,
 2889: 1,
 3061: 1,
 3167: 1,
 3177: 1,
 3271: 1,
 3277: 1,
 3365: 1,
 3411: 1,
 3423: 1,
 3536: 1,
 3745: 1,
 3815: 1,
 3939: 1,
 4035: 1,
 4101: 1,
 4114: 1,
 4175: 1,
 4194: 1,
 4306: 1,
 4339: 1,
 4374: 1,
 4381: 1,
 4390: 1,
 4399: 1,
 4450: 1,
 4505: 1,
 4620: 1,
 4691: 1,
 4789: 1,
 4918: 1,
 5025: 1,
 5104: 1,
 5106: 1,
 5190: 1,
 5192: 1,
 5275: 1,
 5401: 1,
 5414: 1,
 5564: 1,
 5671: 1}

In [10]:
# show it
selected = list(y.keys())[0]
worst = matches[selected]


cv2.imshow("ref", cv2.imread(hashs[selected]["fname"]))
for i in worst:
    # cv2.imshow("face", recognizer._preprocess(faces[i]["face"].image, faces[i]["face"].raw_data))
    cv2.imshow("face", cv2.imread(hashs[i]["fname"]))
    if cv2.waitKey(0) == 27:
        break
cv2.destroyAllWindows()